In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import regex as re

## Getting UH Codes & Names

In [3]:
hidr_raw = pd.DataFrame(
    {'id': [200, 199, 198, 196],
     'id_jusante': ['199 - S. Goncalo', '198 - Maria Xicunguia', '196 - Boiadeiros', '0 - Nenhum'],
     'prod': [1, 1.1, 1.01, 1.001]
    },
).set_index('id', drop=False)

hidr_raw

,id,id_jusante,prod
id,,,
200,200,199 - S. Goncalo,1.000
199,199,198 - Maria Xicunguia,1.100
198,198,196 - Boiadeiros,1.010
196,196,0 - Nenhum,1.001


In [4]:
hidr = hidr_raw.copy(deep=True)
hidr['id_jusante'] = [ int( re.search( pattern=r'([\d]+)\s\-', string=hidr_raw['id_jusante'][id_] ).group(1) ) for id_ in hidr_raw['id'] ]

hidr

,id,id_jusante,prod
id,,,
200,200,199,1.000
199,199,198,1.100
198,198,196,1.010
196,196,0,1.001


## Building a Directional Graph for the Flows 

In [5]:
G = nx.from_pandas_edgelist(
    df=hidr,
    source='id',
    target='id_jusante',
    create_using=nx.DiGraph(),
)

G.nodes

NodeView((200, 199, 198, 196, 0))

In [6]:
prod = hidr['prod'].to_dict()
prod[0] = 0.0

In [7]:
nx.set_node_attributes(
    G, 
    values=prod,
    name='prod',
)

display( G.nodes[198]['prod'], G.nodes[0]['prod'] )

1.01

0.0

In [ ]:
G.nodes

## Cumsum upstream properties

In [8]:
uh_nodes = list(G)
uh_nodes.remove(0)  # Node 0 (ocean), does not count

In [ ]:
for ref_node in uh_nodes:
    prod_acum = ( sum(G.nodes[n]['prod'] for n in nx.ancestors(G, ref_node) | {ref_node}) )
    G.nodes[ref_node]['prod_acum'] = prod_acum
    
display( G.nodes[200]['prod_acum'], G.nodes[198]['prod_acum'] )

In [ ]:
hidr['prod_acum'] = [ G.nodes[id_]['prod_acum'] for id_ in hidr['id'] ]

hidr

In [12]:
## MISC: learning NetworkX

1.0

3.11

In [13]:
hidr['prod_acum'] = [ G.nodes[id_]['prod_acum'] for id_ in hidr['id'] ]

hidr

,id,id_jusante,prod,prod_acum
id,,,,
200,200,199,1.000,1.000
199,199,198,1.100,2.100
198,198,196,1.010,3.110
196,196,0,1.001,4.111


## MISC: learning NetworkX

In [9]:
nx.ancestors(G, source=198)

{199, 200}

In [10]:
hidr['prod'].to_dict()

{200: 1.0, 199: 1.1, 198: 1.01, 196: 1.001}

In [11]:
list(G)

[200, 199, 198, 196, 0]

In [12]:
a = list(G)
a.remove(0)

display(a)

[200, 199, 198, 196]